# Creating a database table from a file containing neuron ids

This notebook contains code to generate a data base table from scratch, using only a list of IDs from a txt file and the available information stored in the server / you data path

In [ ]:
import os
import pandas as pd
import numpy as np
from fafbseg import flywire
from caveclient import CAVEclient
client = CAVEclient('flywire_fafb_production')

## A) Choosing file, loading ids and creating data base table


### Chossing the txt file of interest and id update

In [ ]:
#Loading ids
dataPath = r'Z:\Further projects\Heterogeneity across cell types\data\Txts\cell_type_poofread'
fileName = 'root_ids_Mi1_R_20231106.txt'
filePath = os.path.join(dataPath,fileName)
ids_df = pd.read_csv(filePath, sep = ",")
initial_ids_list = ids_df.columns.tolist()

In [ ]:
#Updating ids
ids_update_df = flywire.update_ids(initial_ids_list, stop_layer=2, supervoxels=None, timestamp=None, dataset='production', progress=True)

In [ ]:
#Filtering out bad updates
filtered_id_update_df = ids_update_df[ids_update_df['confidence'] >= 0.95]
ids_list = filtered_id_update_df['old_id'].tolist()
updated_ids_list = filtered_id_update_df['new_id'].tolist()

In [ ]:
# Getting id identification
identification_df = flywire.find_celltypes(updated_ids_list, user=None, exact=False, case=False, regex=True, update_roots=False)
identification_df.head()

## Creating the data base table

In [ ]:
# Initializing the data set from a dictionary
initial_dict = {}
initial_dict['seg_id'] = identification_df['pt_root_id'].astype(str).tolist()
initial_dict['author'] = identification_df['user_id'].astype(str).tolist()
initial_dict['name'] = identification_df['tag'].astype(str).tolist()
df = pd.DataFrame(initial_dict)



## Saving table in an excel file

In [ ]:
#Saving in a new file

df = df.astype(str)
import datetime
x = datetime.datetime.now()
date_str = x.strftime("%d") + x.strftime("%b") + x.strftime("%Y")
file_name = f'new_table_{fileName}_{date_str}.xlsx'
savePath = os.path.join(dataPath, file_name)
df.to_excel(savePath, sheet_name='Initial table')